<a href="https://colab.research.google.com/github/haruka-inb/pytorch_practice/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='/../../data', train=True,
                                      download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='/../../data', train=False,
                                     download=True, transform=transform)

# create data loader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                           shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

# CNN ((conv->bacthnorm->relu->maxpool)*2 -> 1 fc)
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Linear(7*7*32, num_classes)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1)
    out = self.fc(out)

    return out

model = CNN().to(device)

# load optimizer (crossentropy & adam)
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# train the model
for e in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images, labels = images.to(device), labels.to(device)

  # forward pass
    outputs = model(images)
    loss = criteria(outputs, labels)

  # backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print("Epoch {}/{}, Steps {}/{} training loss is {}"
      .format(e+1, num_epochs, i+1, len(train_loader), loss))

# test the model
correct, total = 0, 0
model.eval()
with torch.no_grad():
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, pred = torch.max(outputs, 1)
    correct += (pred == labels).sum().item()
    total += labels.size(0)

  print("Accuracy is {} %".format(100 * correct / total))

# save the model checkpoint
torch.save(model.state_dict(), "params.ckpt")

Epoch 1/5, Steps 100/600 training loss is 0.08170472085475922
Epoch 1/5, Steps 200/600 training loss is 0.12109977006912231
Epoch 1/5, Steps 300/600 training loss is 0.050953902304172516
Epoch 1/5, Steps 400/600 training loss is 0.05219012871384621
Epoch 1/5, Steps 500/600 training loss is 0.10555092990398407
Epoch 1/5, Steps 600/600 training loss is 0.015571954660117626
Epoch 2/5, Steps 100/600 training loss is 0.1340479999780655
Epoch 2/5, Steps 200/600 training loss is 0.06178664043545723
Epoch 2/5, Steps 300/600 training loss is 0.037592168897390366
Epoch 2/5, Steps 400/600 training loss is 0.05069528520107269
Epoch 2/5, Steps 500/600 training loss is 0.01450662687420845
Epoch 2/5, Steps 600/600 training loss is 0.005234688986092806
Epoch 3/5, Steps 100/600 training loss is 0.01827274262905121
Epoch 3/5, Steps 200/600 training loss is 0.10132969170808792
Epoch 3/5, Steps 300/600 training loss is 0.03023228421807289
Epoch 3/5, Steps 400/600 training loss is 0.01418400276452303
Epoch